# 如何直接将多模态数据传递给模型
在此我们演示如何将[多模态](/docs/concepts/multimodality/)输入直接传递给模型。我们目前预期所有输入都按照[OpenAI要求的格式](https://platform.openai.com/docs/guides/vision)进行传递。对于其他支持多模态输入的模型提供商，我们已在类内部添加了逻辑以转换为预期格式。
在这个示例中，我们将要求一个[模型](/docs/concepts/chat_models/#multimodality)描述一张图片。

In [1]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

In [2]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

最常用的传入图像方式是将其作为字节字符串传入。这应该适用于大多数模型集成。

In [3]:
import base64

import httpx

image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

In [4]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = model.invoke([message])
print(response.content)

The weather in the image appears to be clear and pleasant. The sky is mostly blue with scattered, light clouds, suggesting a sunny day with minimal cloud cover. There is no indication of rain or strong winds, and the overall scene looks bright and calm. The lush green grass and clear visibility further indicate good weather conditions.


我们可以直接在类型为"image_url"的内容块中填入图片URL。需要注意的是，并非所有模型提供商都支持此功能。

In [5]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

The weather in the image appears to be clear and sunny. The sky is mostly blue with a few scattered clouds, suggesting good visibility and a likely pleasant temperature. The bright sunlight is casting distinct shadows on the grass and vegetation, indicating it is likely daytime, possibly late morning or early afternoon. The overall ambiance suggests a warm and inviting day, suitable for outdoor activities.


我们也可以传入多张图片。

In [6]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "are these two images the same?"},
        {"type": "image_url", "image_url": {"url": image_url}},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

Yes, the two images are the same. They both depict a wooden boardwalk extending through a grassy field under a blue sky with light clouds. The scenery, lighting, and composition are identical.


## 工具调用
部分多模态模型也支持[工具调用](/docs/concepts/tool_calling)功能。要使用这类模型调用工具，只需按照[常规方式](/docs/how_to/tool_calling)将工具绑定至模型，并通过指定类型的内容块（例如包含图像数据的内容块）来调用模型。

In [8]:
from typing import Literal

from langchain_core.tools import tool


@tool
def weather_tool(weather: Literal["sunny", "cloudy", "rainy"]) -> None:
    """Describe the weather"""
    pass


model_with_tools = model.bind_tools([weather_tool])

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)

[{'name': 'weather_tool', 'args': {'weather': 'sunny'}, 'id': 'call_BSX4oq4SKnLlp2WlzDhToHBr'}]
